In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

# input and output

In [2]:
###input
cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_re/'
yeast870_path =  '../../../Data/model/yeast-GEM902.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast_genome_file = '../../../Data/Saccharomyces_cerevisiae.fasta'
rxndb2model_compouns_protein_pair_df_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df.csv'
rxndb2model_compouns_protein_pair_df_drop_esp_result_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df_drop_esp.csv'

## output
rxndb_total_info_to_model_espfiltergene_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_espfiltergene.csv'

In [3]:
rxndb2model_compouns_protein_pair_df_drop_esp_result = pd.read_csv(rxndb2model_compouns_protein_pair_df_drop_esp_result_file)
rxndb2model_compouns_protein_pair_df_drop_esp_result.head(3)

result_dict = {}
for index,row in tqdm(rxndb2model_compouns_protein_pair_df_drop_esp_result.iterrows()):
    result_dict[row['Smiles']+ '__' + row['gene']] = row['ESP_Prediction']

180401it [00:09, 18804.79it/s]


In [4]:
rxndb2model_compouns_protein_pair_df = pd.read_csv(rxndb2model_compouns_protein_pair_df_file)
rxndb2model_compouns_protein_pair_df.head(3)


# 定义一个函数来计算esp_score
def calculate_esp_score(row, result_dict):
    key = f"{row['SMILES']}__{row['gene']}"
    return result_dict.get(key, None)  # 如果键不存在，则返回None或其他默认值

# 使用apply方法应用函数到每一行
rxndb2model_compouns_protein_pair_df['esp_score'] = rxndb2model_compouns_protein_pair_df.apply(
    lambda row: calculate_esp_score(row, result_dict), axis=1
)

# 显示前3行以检查结果
rxndb2model_compouns_protein_pair_df.head(3)

,NO,SMILES,gene,esp_score
0,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YLL057C,0.147008
1,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YDR402C,0.122151
2,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YOL114C,0.005617


In [5]:
print(rxndb2model_compouns_protein_pair_df.shape)
rxndb2model_compouns_protein_pair_df = rxndb2model_compouns_protein_pair_df[~rxndb2model_compouns_protein_pair_df['esp_score'].isna()]
print(rxndb2model_compouns_protein_pair_df.shape)

(750521, 4)
(703838, 4)


In [6]:
rxndb2model_compouns_protein_pair_df.head(3)

,NO,SMILES,gene,esp_score
0,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YLL057C,0.147008
1,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YDR402C,0.122151
2,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YOL114C,0.005617


## filter smiles with carbon

In [7]:
rxndb2model_compouns_protein_pair_df = rxndb2model_compouns_protein_pair_df[rxndb2model_compouns_protein_pair_df['SMILES'].apply(smiles_has_carbon)]

In [9]:
import pandas as pd

# 假设你已经加载了 rxndb2model_compouns_protein_pair_df DataFrame

# 初始化结果字典
rxndb_filter_gene = {}

# 遍历每一个唯一的 rxndbID
for rxndbID in tqdm(rxndb2model_compouns_protein_pair_df['NO'].unique()):
    # 获取当前 rxndbID 对应的所有行
    tmp = rxndb2model_compouns_protein_pair_df[rxndb2model_compouns_protein_pair_df['NO'] == rxndbID]
    
    gene_list = []
    
    # 对于每一个唯一的 SMILES
    for met in tmp['SMILES'].unique():
        # 获取当前 SMILES 对应的所有行，并按照 'esp' 列降序排列
        tmp_tmp = tmp[tmp['SMILES'] == met].sort_values(by='esp_score', ascending=False)
        
        # 取前10个基因（如果有那么多）
        top_genes = tmp_tmp['gene'].head(10).tolist()
        gene_list.extend(top_genes)
    
    # 使用 set 去重并转回 list
    rxndb_filter_gene[rxndbID] = list(set(gene_list))

# 打印结果以检查
print(rxndb_filter_gene)

100%|██████████| 7808/7808 [05:21<00:00, 24.30it/s]

{'rxn19747': ['YOL114C', 'YCL059C', 'YDR402C', 'YDR534C', 'YLL057C', 'YHR176W', 'YLR281C', 'YBL098W', 'YBR298C', 'YNR053C'], 'rxn19748': ['YDR402C', 'YDR534C', 'YAL064W', 'YJL151C', 'YCL001W', 'YGR175C', 'YHR176W', 'YBL098W', 'YBR298C', 'YHR007C'], 'rxn19749': ['YDR402C', 'YDR534C', 'YAL064W', 'YJL151C', 'YCL001W', 'YGR175C', 'YHR176W', 'YBL098W', 'YBR298C', 'YHR007C'], 'rxn19750': ['YDR402C', 'YDR534C', 'YAL064W', 'YJL151C', 'YGR175C', 'YHR176W', 'YBL098W', 'YBR298C', 'YNR053C', 'YHR007C'], 'rxn19772': ['YER086W', 'YKL218C', 'YCL064C', 'YGL196W'], 'rxn19793': ['YBR213W', 'YGL012W', 'YNR015W', 'YML080W', 'YDL015C', 'YGL080W', 'YNL202W', 'YLR401C', 'YNL280C', 'YBR026C'], 'rxn19807': ['YER175C', 'YBR034C', 'YDL200C', 'YIL064W', 'YER091C', 'YOL141W', 'YDL212W', 'YOL124C', 'YGL186C', 'YBR213W', 'YKR069W', 'YJR044C', 'YCR024C-A', 'YLR285W', 'YHL036W', 'YLR172C', 'YOR074C', 'YNL092W', 'YLR137W', 'YKL162C'], 'rxn19813': ['YDR343C', 'YMR011W', 'YDR342C', 'YHR094C', 'YGL077C', 'YFL011W', 'YDR34

In [10]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_total_info_to_model.head(1)

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn19747,1.14.13,MNXR180996_reverse,COc1cc(CC(=O)[O-])ccc1O.O>>COc1cc(C(C)=O)ccc1O...,COc1cc(CC(=O)[O-])ccc1O.O.NC(=O)c1ccc[n+]([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'sn_256': -1, 's_0803': -1, 's_1198': -1, 'sn...",sn_256 + s_0803 + s_1198 => sn_226 + s_12...


In [11]:
# 导入tqdm以显示进度条
from tqdm import tqdm

# 设置pandas使用tqdm进度条
tqdm.pandas(desc="Processing rows")

# 使用lambda表达式和progress_apply来更新'GPR'列，不定义额外的函数
rxndb_total_info_to_model['GPR'] = rxndb_total_info_to_model.progress_apply(
    lambda row: '(' + ') or ('.join(rxndb_filter_gene[row['NO']]) + ')' if row['NO'] in rxndb_filter_gene else row['GPR'], 
    axis=1
)

Processing rows: 100%|██████████| 7889/7889 [00:00<00:00, 78704.12it/s]


In [12]:
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_espfiltergene_path,index=None)